In [12]:
from pandas import DataFrame, Series
import pandas as pd
import glob
import re
from tqdm import tqdm
import torchaudio
import numpy as np
import itertools
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import pickle
import os
import glob
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from ast import literal_eval
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import Input 
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from keras.models import load_model

In [13]:
def split_wav(data, sample_rate, start, end):
    start *= sample_rate
    end *= sample_rate
    return data[int(start):int(end)]

In [14]:

sr = 44100
path = "C:/Users/Ahn/projects/upsing/copy_song_mr_removed/"
filenames = os.listdir(path)
song = pd.read_csv("C:/Users/Ahn/projects/upsing/song.csv",converters={"sec":literal_eval})
audio, label = [], []
for file in filenames:
    data, rate = librosa.load(path + file, sr=sr)
    for idx in song.loc[song['song_name'] == file[:-4]].index:
        if song.loc[idx,'label'] == '두성' or song.loc[idx,'label'] == '발음' : 
            continue
        label.append(song.loc[idx,'label'])
        edited_data = split_wav(data, sr, song.loc[idx,'sec'][0], song.loc[idx,'sec'][1])
        if len(edited_data) < 67571:
            edited_data = np.append(edited_data,[0 for _ in range(67571 - len(edited_data))] )
        elif len(edited_data) > 67571:
            edited_data = edited_data[:67571]
        audio.append(np.array(edited_data))
audio, label = np.array(audio), np.array(label)
    # df = pd.DataFrame({'audio' : audio, "label" : label})


In [15]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import random

train_audio, test_audio, train_label, test_label = train_test_split(audio, label, test_size = 0.33, random_state = 2023, shuffle = False)

In [16]:
print(len(train_label))

1539


In [17]:
def make_mel(audio, label, name, save = True):
    with tf.device('/GPU:0'):
      audio_melspectrogram = []
      for x in audio :
        ret = librosa.feature.melspectrogram(y = x, sr = 44100)
        audio_melspectrogram.append(ret)

      amp_db = []
      for i in range(len(audio_melspectrogram)) :
        amp = librosa.amplitude_to_db(np.abs(audio_melspectrogram[i])) ## np.abs?
        amp_db.append(amp)

      features = []
      for i in range(len(amp_db)) :
        features.append([amp_db[i], label[i]])
    
      if save:
        save_pickle(features, f"mel_{name}_data")

      return pd.DataFrame(features, columns = ['audio', 'label'])
train_mel = make_mel(train_audio, train_label, name = "nonaugmented_train", save = False)
test_mel = make_mel(test_audio, test_label, name = "nonaugmented_test", save = False)

In [18]:
teq = ["비브라토", "보컬프라이", "벨팅"]
voc = ["믹스드보이스", "흉성", "가성", "비성"]

In [19]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import random

def train_test(train_features, test_features, shuffle = True, wave_flag=False, labelEncoder = True):
  x_train = np.array(train_features.audio.tolist())
  y_train = np.array(train_features.label.tolist())
  x_test = np.array(test_features.audio.tolist())
  y_test = np.array(test_features.label.tolist())

  if labelEncoder:
    le = LabelEncoder()
    y_train = to_categorical(le.fit_transform(y_train))
    y_test = to_categorical(le.transform(y_test))
    print(le.classes_)

  
  if shuffle: 
    tmp = list(zip(x_train, y_train))
    random.shuffle(tmp)
    x_train, y_train = zip(*tmp)

    tmp1 = list(zip(x_test, y_test))
    random.shuffle(tmp1)
    x_test, y_test = zip(*tmp1)
    x_train, y_train, x_test, y_test = np.array(x_train,dtype=np.float32), np.array(y_train,dtype=np.float32), np.array(x_test,dtype=np.float32), np.array(y_test,dtype=np.float32)
  
  if wave_flag:
    print(x_train.shape)
    x_train = tf.reshape(x_train.astype(float), [-1, x_train.shape[1], 1]) ### 확인
    x_test = tf.reshape(x_test.astype(float), [-1,  x_train.shape[1], 1])
    return x_train, y_train, x_test, y_test, le
  
  n_channels = 1

  with tf.device('/gpu:0'):
    x_train = tf.reshape(x_train, [-1, x_train.shape[1], x_train.shape[2], n_channels])
    x_test = tf.reshape(x_test, [-1,  x_test.shape[1], x_test.shape[2], n_channels])


  return x_train, y_train, x_test, y_test, le

In [20]:
x_train, y_train, x_test, y_test, le = train_test(train_mel, test_mel)

['가성' '믹스드보이스' '벨팅' '보컬프라이' '비브라토' '비성' '흉성']


In [21]:
# (None, 127, 86, 16) 
def make_model(input):
  model = keras.Sequential()

  model.add(layers.Conv2D(input_shape=(input[0].shape[0], input[0].shape[1], 1), filters=16, kernel_size=(1,2), activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=2))
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(kernel_size=(1,8), filters=32, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=2))
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(kernel_size=(2,1), filters=64, activation='relu')) # 64
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=3))
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv2D(kernel_size=(8,1), filters=128, activation='relu')) # 128
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=2)) ##
  model.add(layers.Dropout(0.3)) #3

  # model.add(layers.GlobalAveragePooling2D())
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(units=64)) # 128
  model.add(tf.keras.layers.Dense(units=22))
  model.add(tf.keras.layers.Dense(units=7, activation='softmax'))

  model.summary()
  return model

training_epochs = 100
num_batch_size = 64
learning_rate = 1e-4

In [22]:
tmp_model = make_model(x_train)
tmp_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=learning_rate), metrics=['accuracy'])
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
with tf.device('/gpu:0'):
    tmp_model.fit(x_train, y_train, batch_size = num_batch_size, epochs = training_epochs , validation_data = (x_test, y_test) , callbacks = [early])
    results = tmp_model.evaluate(x_test, y_test, batch_size=64)
    print('test loss, test acc:', results)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 128, 131, 16)      48        
                                                                 
 batch_normalization_4 (Batc  (None, 128, 131, 16)     64        
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 64, 65, 16)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 64, 65, 16)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 58, 32)        4128      
                                                                 
 batch_normalization_5 (Batc  (None, 64, 58, 32)      

KeyboardInterrupt: 

In [2]:
!conda install tensorflow-gpu

^C


In [ ]:
!pip install numpy==1.20.3